# Draco Debugging

In [1]:
import toml
from draco import Draco
from draco.debug import DracoDebug, DracoDebugPlotter

## Why doesn't Draco produce what I want?

There are several reasons:

First, the generator in `generate.lp` may not be capable of enumerate every possibility. Then, the answer you have in mind might also have been left out by hard constraints. An extreme situation you might observe is that Draco returns an [empty result](#debugging-empty-results). If you have edge cases you want to test on, you can add constraints temporarily to check if Draco can still generate answers. For how to do this, refer to *[How to modify the knowledge-base with constraints](#how-to-modify-the-knowledge-base-with-constraints)*. Then you can extend the generator or revise the hard constraints accordingly after making sure your testing cases are well-defined.

If you have pinpointed the issue to the hard constraints collection, you might want to take a step to relax the hard constraints for further investigation before you make any changes. For how to do this, refer to *[Debugging empty results](#debugging-empty-results)*.

Second, the answer might have a low ranking so that Draco promotes other answers instead. This can be caused by lack of proper soft constraints, incorrectly-defined soft constraints, or unsuitable constraint weights. To make sure the soft constraints are well-defined, add more unit tests and refer to *[Validating the soft constraints with full specifications](#validating-the-soft-constraints-with-full-specifications)*. For soft constraint introspection, check out *[How to modify the knowledge-base with constraints](#how-to-modify-the-knowledge-base-with-constraints)*.

In general, here is a guideline for what to do:
* If you see that there are too many recommendations, you can:
  * add more hard constraints
  * modify your generator and hard constraints to reduce symmetry in the search space (e.g. similar recommendations with switched entity ids)


* If you see too few recommendations, you can:
  * check if some of your constraints are too tight, and move them to soft constraints
 

* If you see no recommendations, you might have made mistakes in the hard constraints. You can allow violations to check what are the common ones by removing the `violation` constraint, which forbids any violations, from the programs. 

## Validating the soft constraints with full specifications

Although we ensure 100% testing coverage including all soft constraints, we might want to assess and validate that the soft constraint collection carries out the intended goal in real-world visualization designs. 

Here we demonstrate how we validate the soft constraints violated in specific examples.

### Loading Specifications

Specifications to debug can be declared as a dictionary or can be loaded from a static file. The debugger will generate a dataframe from your input automatically, ready to be analyzed!

In [2]:
specs = toml.load(open("./data/example_charts.toml"))
default_draco = Draco()

debugger = DracoDebug(specs=specs, draco=default_draco)
chart_preferences = debugger.chart_preferences
chart_preferences.head()

,chart_name,pref_name,count,weight
0,tick_plot,linear_x,1,0
1,tick_plot,c_d_overlap_tick,1,0
2,tick_plot,linear_scale,1,0
3,tick_plot,continuous_pos_not_zero,1,1
4,tick_plot,continuous_not_zero,1,1


### Visualizing Debug Data

The `chart_preferences` dataframe generated by `DracoDebug` can be visualized using `DracoDebugPlotter`. Custom plot configurations can be passed to `DracoDebugPlotter.create_chart` to customize the produced visualization.

In [3]:
plotter = DracoDebugPlotter(chart_preferences)
# Creates a chart using the default config (alphabetical sort)
plotter.create_chart()

alt.VConcatChart(...)

### Interactive Selection of Debugging Variants

_Interactions will work only in a Python-enabled environment! Please [clone the repository](https://github.com/cmudig/draco2) or start it in [Binder](https://mybinder.org/v2/gh/cmudig/draco2/HEAD)._

In [4]:
import ipywidgets as widgets
from IPython.display import clear_output

configs = DracoDebugPlotter.__DEFAULT_CONFIGS__
DEFAULT_CFG = configs[1]
chart_output: widgets.Output | None

config_selector = widgets.Dropdown(
    options=[cfg.title for cfg in configs],
    value=DEFAULT_CFG.title,
    description="Sorting:",
    disabled=False,
)


def on_config_selected(cfg_title: str):
    """Generates and displays a chart for the config identified by `cfg_title`"""
    cfg = [cfg for cfg in configs if cfg.title == cfg_title]
    if len(cfg) == 0:
        raise RuntimeError(f'No chart configuration with the title "{cfg_title}"')
    chart = plotter.create_chart(cfg[0])
    clear_output()
    display(config_selector)
    display(chart)


def handle_config_selection(event):
    """Handler registered to the `config_selector` dropdown widget"""
    if event["type"] == "change":
        cfg_title: str = event["owner"].value
        on_config_selected(cfg_title)


# Register the event handler to the dropdown
config_selector.observe(handle_config_selection)
# Display initial value
on_config_selected(DEFAULT_CFG.title)

Dropdown(description='Sorting:', index=1, options=('Sort alphabetically', 'Sort by count sum'), value='Sort by…

alt.VConcatChart(...)

With the heatmap above, we can verify if each soft constraint expresses the intended rule under the context of  visualization designs. In the collection above, there are groups of visualizations that are more relevant to each other in terms of their specification. For example, knowing that the most significant difference between a `bubble chart` and a `colored scatterplot` is that a `bubble chart` has a quantitative `size` channel while a `colored scatterplot` has a categorical `color` channel, one can verify that the relevant constraints are violated accordingly (i.e., `categorical_scale` and `categorical_color` for the `colored scatterplot`, and the `size_not_zero` and `linear_size` for the `bubble chart`). 

In addition, the heatmap indicates the coverage of violations in the chosen example. We can then incrementally add new examples that focus on constraints with low/zero coverage. 

On the other hand, when we identify that the existing constraints saturated for substantially different example, we might be able to uncover new constraints to extend the knowledge space. 

## How to modify the knowledge base with constraints

You can design your own description language and use it with Draco or extend the existing language we use here. If you don't know where to start with the constraints, you can first use our Draco APIs to generate some recommendations. Then, you should be able to find some recommendations that should have been left out, and you can write constraints to reflect them.
If you write your own description language, you need to set up the search space in a similar way to `generate.lp` before trying to generate recommendations.

For example, the following snippet shows how to generate 5 recommendation. You can set a different number to look into more results.

In [5]:
from draco import Draco, answer_set_to_dict, dict_to_facts
from pprint import pprint


d = Draco()

In [6]:
from draco.renderer import AltairRenderer
from vega_datasets import data
from IPython.display import display
import warnings

# Suppressing warnings raised by altair in the background
# (iteration-related deprecation warnings)
warnings.filterwarnings("ignore")

# Setting up renderer and demo data
renderer = AltairRenderer()
weather_data = data.seattle_weather()

In [7]:
input_spec = [
    "attribute(task,root,summary).",
    "attribute(number_rows,root,100).",
    "entity(field,root,temp_max).",
    "attribute((field,name),temp_max,temp_max).",
    "attribute((field,type),temp_max,number).",
    "attribute((field,unique),temp_max,100).",
    "entity(field,root,f1).",
    "attribute((field,name),f1,precipitation).",
    "attribute((field,type),f1,number).",
    "entity(view,root,(v,0)).",
    "entity(mark,(v,0),(m,0)).",
    "entity(encoding,(m,0),(e,0)).",
    "attribute((encoding,field),(e,0),temp_max).",
    "entity(encoding,(m,0),(e,1)).",
    "attribute((encoding,field),(e,1),precipitation).",
    "#show entity/3.",
    "#show attribute/3.",
]

print("INPUT:")
print(input_spec)

print("OUTPUT:")

specs = {}

for i, model in enumerate(d.complete_spec(input_spec, 5)):
    chart_num = i + 1
    spec = answer_set_to_dict(model.answer_set)
    chart_name = f"Rec {chart_num}"
    specs[chart_name] = dict_to_facts(spec)
    print(f"CHART {chart_num}")
    print(f"COST: {model.cost}")
    pprint(spec)
    pprint(str(model))
    display(renderer.render(spec=spec, data=weather_data))

print("VIOLATED PREFERENCES:")
debugger = DracoDebug(specs=specs, draco=default_draco)
chart_preferences = debugger.chart_preferences
plotter = DracoDebugPlotter(chart_preferences)

# sort by sum of count
plotter.create_chart(DracoDebugPlotter.__DEFAULT_CONFIGS__[1])

INPUT:
['attribute(task,root,summary).', 'attribute(number_rows,root,100).', 'entity(field,root,temp_max).', 'attribute((field,name),temp_max,temp_max).', 'attribute((field,type),temp_max,number).', 'attribute((field,unique),temp_max,100).', 'entity(field,root,f1).', 'attribute((field,name),f1,precipitation).', 'attribute((field,type),f1,number).', 'entity(view,root,(v,0)).', 'entity(mark,(v,0),(m,0)).', 'entity(encoding,(m,0),(e,0)).', 'attribute((encoding,field),(e,0),temp_max).', 'entity(encoding,(m,0),(e,1)).', 'attribute((encoding,field),(e,1),precipitation).', '#show entity/3.', '#show attribute/3.']
OUTPUT:
CHART 1
COST: [12]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'channel': 'x', 'field': 'temp_max'},
                                  {'channel': 'y', 'field': 'precipitation'}],
    

alt.Chart(...)

CHART 2
COST: [12]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'channel': 'y', 'field': 'temp_max'},
                                  {'channel': 'x', 'field': 'precipitation'}],
                     'type': 'point'}],
           'scale': [{'channel': 'y', 'type': 'linear', 'zero': 'true'},
                     {'channel': 'x', 'type': 'linear', 'zero': 'true'}]}]}
('attribute(task,root,summary).\n'
 'attribute(number_rows,root,100).\n'
 'attribute((field,name),temp_max,temp_max).\n'
 'attribute((field,type),temp_max,number).\n'
 'attribute((field,unique),temp_max,100).\n'
 'attribute((field,name),f1,precipitation).\n'
 'attribute((field,type),f1,number).\n'
 'attribute((encoding,field),(e,0),temp_max).\n'
 'attribute((encoding,field),(e,1),precipitation).\n'
 'attribute((encoding,channel),(e,1

alt.Chart(...)

CHART 3
COST: [13]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'channel': 'y', 'field': 'temp_max'},
                                  {'channel': 'x', 'field': 'precipitation'}],
                     'type': 'point'}],
           'scale': [{'channel': 'y', 'type': 'linear', 'zero': 'true'},
                     {'channel': 'x', 'type': 'log', 'zero': 'true'}]}]}
('attribute(task,root,summary).\n'
 'attribute(number_rows,root,100).\n'
 'attribute((field,name),temp_max,temp_max).\n'
 'attribute((field,type),temp_max,number).\n'
 'attribute((field,unique),temp_max,100).\n'
 'attribute((field,name),f1,precipitation).\n'
 'attribute((field,type),f1,number).\n'
 'attribute((encoding,field),(e,0),temp_max).\n'
 'attribute((encoding,field),(e,1),precipitation).\n'
 'attribute((encoding,channel),(e,1),x

...

alt.Chart(...)

CHART 5
COST: [14]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'y',
                                   'field': 'temp_max'},
                                  {'channel': 'x', 'field': 'precipitation'}],
                     'type': 'tick'}],
           'scale': [{'channel': 'y', 'type': 'linear'},
                     {'channel': 'x', 'type': 'linear', 'zero': 'true'}]}]}
('attribute(task,root,summary).\n'
 'attribute(number_rows,root,100).\n'
 'attribute((field,name),temp_max,temp_max).\n'
 'attribute((field,type),temp_max,number).\n'
 'attribute((field,unique),temp_max,100).\n'
 'attribute((field,name),f1,precipitation).\n'
 'attribute((field,type),f1,number).\n'
 'attribute((encoding,field),(e,0),temp_max).\n'
 'attribute((encoding

alt.Chart(...)

VIOLATED PREFERENCES:


alt.VConcatChart(...)

We can add hard constraints temporarily to further filter and inspect the answer set: 

In [8]:
# exclude multi-layer or multi-view designs.
# filter out designs with less than 3 encodings
new_input = input_spec + [
    ":- {entity(encoding,_,_)} <= 2.",
    ":- {entity(mark,_,_)} >= 2.",
]

We will demonstrate how we found a bug in one of our soft constraints and fixed it. 
At the beginning, the soft constraint `binned_orientation_not_x` was incorrectly defined as:
```prolog
preference(binned_orientation_not_x,E) :- 
    attribute((field,type),F,(number;datetime)), 
    helper((encoding,field),E,F), 
    attribute((encoding,binning),E,_), 
    not attribute((encoding,channel),E,x).
```
Now, to simulate our debugging process, let's replace the current correct definition with it:

In [9]:
from draco.asp_utils import blocks_to_program
from draco.programs import soft

# remove the correct definition, and then replace it with the incorrect one.
s = "".join(
    blocks_to_program(
        soft.blocks, set(soft.blocks.keys()) - {"binned_orientation_not_x"}
    )
)

s += "preference(binned_orientation_not_x,E) :- \
    attribute((field,type),F,(number;datetime)), \
    helper((encoding,field),E,F), \
    attribute((encoding,binning),E,_), \
    not attribute((encoding,channel),E,x)."
new_draco = Draco(soft=s)

specs = {}
for i, model in enumerate(new_draco.complete_spec(new_input, 5)):
    chart_num = i + 1
    spec = answer_set_to_dict(model.answer_set)
    chart_name = f"Rec {chart_num}"
    specs[chart_name] = dict_to_facts(spec)
    print(f"CHART {chart_num}")
    print(f"COST: {model.cost}")
    pprint(spec)
    display(renderer.render(spec=spec, data=weather_data))

print("VIOLATED PREFERENCES:")
debugger = DracoDebug(specs=specs, draco=new_draco)
chart_preferences = debugger.chart_preferences
plotter = DracoDebugPlotter(chart_preferences)

# sort by sum of count
plotter.create_chart(DracoDebugPlotter.__DEFAULT_CONFIGS__[1])

CHART 1
COST: [18]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'x',
                                   'field': 'temp_max'},
                                  {'channel': 'y', 'field': 'precipitation'},
                                  {'aggregate': 'count', 'channel': 'size'}],
                     'type': 'point'}],
           'scale': [{'channel': 'x', 'type': 'linear'},
                     {'channel': 'y', 'type': 'ordinal'},
                     {'channel': 'size', 'type': 'linear', 'zero': 'true'}]}]}


alt.Chart(...)

CHART 2
COST: [18]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'x',
                                   'field': 'temp_max'},
                                  {'channel': 'y', 'field': 'precipitation'},
                                  {'aggregate': 'count', 'channel': 'size'}],
                     'type': 'point'}],
           'scale': [{'channel': 'x', 'type': 'linear'},
                     {'channel': 'y', 'type': 'ordinal', 'zero': 'true'},
                     {'channel': 'size', 'type': 'linear', 'zero': 'true'}]}]}


alt.Chart(...)

CHART 3
COST: [18]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'x',
                                   'field': 'temp_max'},
                                  {'channel': 'y', 'field': 'precipitation'},
                                  {'aggregate': 'count', 'channel': 'size'}],
                     'type': 'point'}],
           'scale': [{'channel': 'x', 'type': 'linear', 'zero': 'true'},
                     {'channel': 'y', 'type': 'ordinal'},
                     {'channel': 'size', 'type': 'linear', 'zero': 'true'}]}]}


...

alt.Chart(...)

CHART 5
COST: [19]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'x',
                                   'field': 'temp_max'},
                                  {'binning': 10,
                                   'channel': 'y',
                                   'field': 'precipitation'},
                                  {'aggregate': 'count', 'channel': 'size'}],
                     'type': 'point'}],
           'scale': [{'channel': 'size', 'type': 'linear', 'zero': 'true'},
                     {'channel': 'y', 'type': 'linear'},
                     {'channel': 'x', 'type': 'linear'}]}]}


alt.Chart(...)

VIOLATED PREFERENCES:


alt.VConcatChart(...)

Note that the first four charts shouldn't be preferred because the `precipitation` field with floating numbers shouldn't be treated as discrete.

The difference between them falls on the `bin`, `ordinal_scale` and `binned_orientation_not_x` since they are all non-zero. Then with `bin` and `ordinal_scale` having the same count and weight, the only reason chart 5 was ranked lower was the `binned_oriented_not_x`. However, chart 5 does have binned x-axis. Realizing that `binned_oriented_not_x` should have meant that "Prefer binned quantitative on x-axis **if y-axis is not binned**", we fixed the soft constraint definition in `soft.lp` as:
```prolog
preference(binned_orientation_not_x,E) :- 
    attribute((field,type),F,(number;datetime)), 
    helper((encoding,field),E,F), 
    attribute((encoding,binning),E,_), 
    not attribute((encoding,channel),_,x).
```

In [10]:
specs = {}

for i, model in enumerate(d.complete_spec(new_input)):
    chart_num = i + 1
    spec = answer_set_to_dict(model.answer_set)
    chart_name = f"Rec {chart_num}"
    specs[chart_name] = dict_to_facts(spec)
    print(f"CHART {chart_num}")

    print(f"COST: {model.cost}")
    pprint(spec)
    display(renderer.render(spec=spec, data=weather_data))

print("VIOLATED PREFERENCES:")
debugger = DracoDebug(specs=specs, draco=default_draco)
chart_preferences = debugger.chart_preferences
chart_preferences.head()

CHART 1
COST: [18]
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'x',
                                   'field': 'temp_max'},
                                  {'channel': 'y', 'field': 'precipitation'},
                                  {'aggregate': 'count', 'channel': 'size'}],
                     'type': 'point'}],
           'scale': [{'channel': 'x', 'type': 'linear'},
                     {'channel': 'y', 'type': 'ordinal'},
                     {'channel': 'size', 'type': 'linear', 'zero': 'true'}]}]}


alt.Chart(...)

VIOLATED PREFERENCES:


,chart_name,pref_name,count,weight
0,Rec 1,cartesian_coordinate,1,0
1,Rec 1,summary_point,1,0
2,Rec 1,aggregate_count,1,0
3,Rec 1,ordinal_y,1,0
4,Rec 1,linear_size,1,1


## Debugging empty results

While we are exploring with the knowledge base with partial specifications as input, we may encounter the situation where Draco returns empty results.

If you see too few recommendations, you can check if some of your constraints are too tight, and move them to soft constraints. 
If you see no recommendations, you might have made mistakes in the hard constraints. You can allow violations to check what are the common ones by removing the violation constraint, which forbids any violations, from the programs. Below is an example:

In [11]:
from draco.programs import constraints

c = "".join(
    blocks_to_program(
        constraints.blocks, set(constraints.blocks.keys()) - {"violation"}
    )
)
new_draco = Draco(constraints=c)

for model in new_draco.complete_spec(new_input):
    spec = answer_set_to_dict(model.answer_set)
    pprint(spec)

    print("VIOLATED HARD CONSTRAINTS:")
    answer = [str(symbol) + ". " for symbol in model.answer_set]
    print(new_draco.get_violations(answer))

{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'channel': 'x', 'field': 'temp_max'},
                                  {'channel': 'x', 'field': 'precipitation'},
                                  {'channel': 'y'}],
                     'type': 'rect'}],
           'scale': [{'channel': 'x', 'type': 'linear', 'zero': 'true'},
                     {'channel': 'y', 'type': 'linear', 'zero': 'true'}]}]}
VIOLATED HARD CONSTRAINTS:
['rect_without_d_d', 'encoding_no_field_and_not_count', 'encoding_repeat_channel']
